## Dummy dataframe

For this project I need to create a dummy csv file that has a dataframe. It will consist of about 1000 rows and will contain relevant data and data with errors. This is because I need my parquet conversion program to clean the data before converting it to a parquet file 

In [10]:
import sys
print(sys.path)
import time

['/home/sam_glass/parquet_conversion/dummy_data', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/sam_glass/parquet_conversion/venv/lib/python3.11/site-packages']


In [2]:
import pandas as pd
import numpy as np

/tmp/ipykernel_49047/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# use this later for converting unix to datetime
# pd.set_option("display.float_format", lambda x: time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(x)))

In [3]:
# setting random seed to 42 for consistence with random generated numbers
np.random.seed(42) 

In [21]:
# creating a dataframe with 1000 rows
df = pd.DataFrame(index=range(1000))

# Adding a Unix datetime column with one second increments
start_time = int(time.mktime(time.strptime("2023-01-31 00:00:00", "%Y-%m-%d %H:%M:%S")))

df["Timestamp"] = pd.to_numeric(range(start_time, start_time + 1000))



df["Timestamp"].info()


<class 'pandas.core.series.Series'>
RangeIndex: 1000 entries, 0 to 999
Series name: Timestamp
Non-Null Count  Dtype
--------------  -----
1000 non-null   int64
dtypes: int64(1)
memory usage: 7.9 KB
